In [ ]:
import pandas as pd 
import numpy as np 
import os
import re
from datetime import datetime,timedelta
import math
import calendar

In [ ]:
path = os.getcwd()
datapath = os.path.join(path,"Data")
datapath

In [ ]:
security_id = "500002"
company_name = "ABB India Limited,Active"

In [ ]:
stock = pd.read_csv(os.path.join(datapath,"Stock/"+"mod"+security_id+".csv"))
stock.Date = pd.to_datetime(stock.Date)

In [ ]:
direct_columns = ['Open Price', 'High Price', 'Low Price', 'Close Price','Next Day Open Price', 'Next Day High Price', 'Next Day Low Price', 'Next Day Close Price','WAP','No.of Shares', 'No. of Trades', 'Total Turnover (Rs.)','Deliverable Quantity', '% Deli. Qty to Traded Qty','Spread High-Low','Spread Close-Open','Alpha','Beta']
growth_direct_rate_columns = [col + " GR" for col in direct_columns]

In [ ]:
def find_gain_loss(stock):
    stock[growth_direct_rate_columns] = pd.DataFrame([[0]*len(growth_direct_rate_columns)], index=stock.index)
    today = stock[direct_columns]
    previous = stock[direct_columns].shift(1)
    stock[growth_direct_rate_columns] = (today-previous)/previous
    return stock

In [ ]:
def close_price_as_percent_of_LV_HV_BA(stock):
    bands = [7,30,90,180,365]
    for b in bands:
        bcols = ["CP % LV "+str(b)+" days","CP % HV "+str(b)+" days","CP % BA "+str(b)+" days"]
        stock[bcols] = pd.DataFrame([[0]*len(bcols)], index=stock.index)
        for i in range(stock.shape[0]):
            s = i+1
            specific_bands = stock.iloc[-(s):-(s+b+1):-1]
            low = specific_bands["Close Price"].min()
            high = specific_bands["Close Price"].max()
            today = stock.iloc[-(s)]["Close Price"]
            stock.loc[specific_bands.index,bcols] = [today/low,today/high,today/(high-low)]
    return stock

In [ ]:
def sequential_increase(stock):
    stock["Sequential Increase"] = 0
    c = 1
    for i in reversed(range(stock.shape[0]-1)):
        if stock.at[i,"Close Price"] > stock.at[i+1,"Close Price"]:
            stock.at[i,"Sequential Increase"] = c
            c += 1
        else:
            stock.at[i,"Sequential Increase"] = 0
            c = 1
    return stock

In [ ]:
def sequential_decrease(stock):
    stock["Sequential Decrease"] = 0
    c = 1
    stock.at[stock.shape[0]-2, "Sequential Decrease"] = 0
    stock.at[stock.shape[0]-1, "Sequential Decrease"] = 0
    for i in reversed(range(stock.shape[0]-1)):
        if stock.at[i,"Close Price"] < stock.at[i+1,"Close Price"]:
            stock.at[i,"Sequential Decrease"] = c
            c += 1
        else:
            stock.at[i,"Sequential Decrease"] = 0
            c = 1
    return stock

In [ ]:
def sequential_increase_percentage(stock):
    stock["Sequential Increase %"] = ""
    for i in range(stock.shape[0]-2):
        if stock.at[i, "Sequential Increase"] != 0:
            inc = stock.at[i, "Sequential Increase"]
        else:
            inc = 1
        fr = stock.at[i+1, "Close Price"]
        to = stock.at[i+1+inc, "Close Price"]
        stock.at[i, "Sequential Increase %"] = (fr - to) / to
    stock.at[stock.shape[0]-2, "Sequential Increase %"] = 0
    stock.at[stock.shape[0]-1, "Sequential Increase %"] = 0
    return stock

In [ ]:
def sequential_decrease_percentage(stock):
    stock["Sequential Decrease %"] = ""
    for i in range(stock.shape[0]-2):
        if stock.at[i, "Sequential Decrease"] != 0:
            inc = stock.at[i, "Sequential Decrease"]
        else:
            inc = 1
        fr = stock.at[i+1, "Close Price"]
        to = stock.at[i+1+inc, "Close Price"]
        stock.at[i, "Sequential Decrease %"] = (to - fr) / fr
    stock.at[stock.shape[0]-2, "Sequential Decrease %"] = 0
    stock.at[stock.shape[0]-1, "Sequential Decrease %"] = 0
    return stock

In [ ]:
def max_min_avg_of_sequential_data(stock):
    index_start = stock.first_valid_index() 
    seq_inc_days = stock.at[index_start, "Sequential Increase"]
    seq_dec_days = stock.at[index_start, "Sequential Decrease"]
    seq_inc_list = [0]
    seq_dec_list = [0]
    for i in range(index_start, stock.shape[0]+index_start-1):
        if stock.at[i, "Sequential Increase"] == seq_inc_days:
            seq_inc_list.append(stock.at[i, "Sequential Increase %"])
        if stock.at[i, "Sequential Decrease"] == seq_dec_days:
            seq_dec_list.append(stock.at[i, "Sequential Decrease %"])
    return seq_inc_list, seq_dec_list

In [ ]:
def sequential_increase_decrease(stock):
    bands = [90,180,365]
    for b in bands:
        bcols = ["Max Inc % in "+str(b)+" days","Max Dec % in "+str(b)+" days","Min Inc % in "+str(b)+" days","Min Dec % in "+str(b)+" days","Avg Inc % in "+str(b)+" days","Avg Dec % in "+str(b)+" days"]
        stock[bcols] = pd.DataFrame([[0]*len(bcols)], index=stock.index)
        for i in range(stock.shape[0]):
            s = i+1
            specific_bands = stock.iloc[-(s):-(s+b+1):-1]
            specific_bands.sort_index(inplace=True)
            seq_inc_list, seq_dec_list = max_min_avg_of_sequential_data(specific_bands)
            stock.loc[specific_bands.index,bcols] = [max(seq_inc_list),max(seq_dec_list),min(seq_inc_list),min(seq_dec_list),np.mean(seq_inc_list),np.mean(seq_dec_list)]
    return stock

In [ ]:
def generate_dictionary_for_quarterwise_data(stock,columnName):
    result = {}
    for index,row in stock.iterrows():
        q = (row.Date.month-1)//3 + 1   
        year = row.Date.year
        month = row.Date.month
        res = result.get(year,{})
        # amount = re.findall(r"\d+.?\d*",row["Revenue"])[0]
        amount  = row[columnName]
        q = "1q" if 1 <= month <= 3 else "2q" if 4 <= month <= 6 else "3q" if 6 <= month <= 9 else "4q"
        val = res.get(q,[])
        val.append(float(amount))
        res[q] = val
        result[year] = res
    return result

In [ ]:
def generate_dictionary_for_quarterwise_growthrate_data(data):
    gr_dic = {}
    keys = list(data.keys())
    array = [''] * (len(keys)*4)
    array_index = 0
    for key in data:
        lists = data.get(key)
        array_index += 4 - len(lists.keys()) 
        for lis in lists:
            if math.isnan(lists.get(lis)[0]):
                array[array_index] = ''
            else:                
                array[array_index] = lists.get(lis)[0]
            array_index = array_index + 1
    # if (array.count('')) > ((len(keys) * 4) / 2):
    #     return gr_dic
    
    for i in range(4,len(keys)*4,4):
        res = [array[i],array[i+1],array[i+2],array[i+3]]
        avg = np.mean(list(filter(lambda i: isinstance(i, float), res)))
        if np.isnan(avg):
            pass
        else:
            array[i] = avg


    gr_array = [''] * (len(keys)*4)
    for i in range(0, len(keys)*4-1):
        x = array[i]
        y = array[i+1]
        if x == '' and y == '': continue
        if y == '': continue
        if x == '':
            gr_array[i] = 1
        else:
            gr_array[i] = (x - y) / y
    index = 0
    for key in data:
        gr_dic[key] = [gr_array[index], gr_array[index+1], gr_array[index+2], gr_array[index+3]]
        index = index + 4
    return gr_dic

In [ ]:
def update_growthrate_for_quarterwise_data(gr_dic, stock, columnName):
    for i in range(0, stock.shape[0]-1):
        date = stock.at[i, "Date"]
        q = int((date.month-1)//3)
        year = date.year
        if year in gr_dic.keys():
            stock.at[i,columnName+" GR"] = gr_dic.get(year)[q] if isinstance(gr_dic.get(year)[q],float) else 0
    return stock

In [ ]:
def quarter_wise_growthrate(stock, columnName):
    dic = generate_dictionary_for_quarterwise_data(stock, columnName)
    gr_dic = generate_dictionary_for_quarterwise_growthrate_data(dic)
    stock[columnName + ' GR'] = ''
    if gr_dic == {}:
        return stock
    else:
        stock = update_growthrate_for_quarterwise_data(gr_dic, stock, columnName)
    return stock

In [ ]:
stock = find_gain_loss(stock)
stock = close_price_as_percent_of_LV_HV_BA(stock)

In [ ]:
stock = sequential_increase(stock)
stock = sequential_decrease(stock)

In [ ]:
stock = sequential_increase_percentage(stock)
stock = sequential_decrease_percentage(stock)

In [ ]:
stock = sequential_increase_decrease(stock)

In [ ]:
cols = ["Revenue","Dividend","Income","Expenditure","Net Profit","EPS"]

In [ ]:
for col in cols:
    try:
        stock = quarter_wise_growthrate(stock, col)
    except:
        pass

In [ ]:
stock.to_csv(os.path.join(datapath,"Stock/"+"mod"+security_id+".csv"),index=None)